In [1]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install konlpy


mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [2]:
from collections import Counter
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [8]:
corpus= [
    "나는 나를 사랑한다.",
    "나는 너를 사랑한다.",
    "너는 나를 사랑한다.",
    "너는 너를 사랑한다.",
    "나는 나를 증오한다.",
    "나는 너를 증오한다.",
    "너는 나를 증오한다.",
    "너는 너를 증오한다."
    ]
df=pd.DataFrame({"id": range(len(corpus)),"corpus": corpus,"label":[0,1,1,0,1,0,0,1]
})                 
df



s=" ".join(df.corpus)

print('All string :' ,s)

# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF","IC"]
posToUse=["NNP","NNG","NP","NNB"]

def getTokens(s):
    global posToUse
    return [ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ] 
    
print('All tockents:', getTokens(s))

posToUse=["NNP","NNG","NP","NNB"]
stopwords=["놈","18"]
def getToken(s,pos=posToUse,stopword=stopwords):
    return [ w for w,t in  Mecab().pos(s) if t in pos and w not in stopword ] 
vect = CountVectorizer(tokenizer=getToken)
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)
cd=pd.DataFrame(vect.transform([" ".join(getTokens(i)) for i in df.corpus]).toarray())
data=pd.concat([df,cd], axis=1)
data

All string : 나는 나를 사랑한다. 나는 너를 사랑한다. 너는 나를 사랑한다. 너는 너를 사랑한다. 나는 나를 증오한다. 나는 너를 증오한다. 너는 나를 증오한다. 너는 너를 증오한다.
All tockents: ['나', '나', '사랑', '나', '너', '사랑', '너', '나', '사랑', '너', '너', '사랑', '나', '나', '증오', '나', '너', '증오', '너', '나', '증오', '너', '너', '증오']
Dictionary :  {'나': 0, '사랑': 2, '너': 1, '증오': 3}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


,id,corpus,label,0,1,2,3
0,0,나는 나를 사랑한다.,0,2,0,1,0
1,1,나는 너를 사랑한다.,1,1,1,1,0
2,2,너는 나를 사랑한다.,1,1,1,1,0
3,3,너는 너를 사랑한다.,0,0,2,1,0
4,4,나는 나를 증오한다.,1,2,0,0,1
5,5,나는 너를 증오한다.,0,1,1,0,1
6,6,너는 나를 증오한다.,0,1,1,0,1
7,7,너는 너를 증오한다.,1,0,2,0,1


In [9]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop


x_train=data.iloc[:,3:11].to_numpy()
#y_train=data.label.to_numpy().reshape([-1,1])
y_train=keras.utils.to_categorical(data.label,2)

In [10]:
print(x_train.shape)
print(y_train.shape)

(8, 4)
(8, 2)


In [17]:
model = Sequential()
model.add(Dense(14, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(14, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 14)                70        
_________________________________________________________________
dense_17 (Dense)             (None, 14)                210       
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 60        
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 10        
Total params: 350
Trainable params: 350
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_train, y_train))


Epoch 1/100
8/8 [==============================] - 1s 42ms/step - loss: 0.7381 - accuracy: 0.4635 - val_loss: 0.6983 - val_accuracy: 0.5000
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6491 - accuracy: 0.6616 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6217 - accuracy: 0.7320 - val_loss: 0.6903 - val_accuracy: 0.5000
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 0.7135 - accuracy: 0.5098 - val_loss: 0.6873 - val_accuracy: 0.5000
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 0.7744 - accuracy: 0.2180 - val_loss: 0.6843 - val_accuracy: 0.5000
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7201 - accuracy: 0.3384 - val_loss: 0.6822 - val_accuracy: 0.5000
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.4598 - val_loss: 0.6806 - val_accuracy: 0.5000
Epoch 8/100
8/8 [=========

In [19]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.31695613265037537
Train accuracy: 1.0


In [20]:
model.predict(x_train)

array([[0.5822281 , 0.41777185],
       [0.09134443, 0.9086555 ],
       [0.09134443, 0.9086555 ],
       [0.58360505, 0.41639492],
       [0.04711778, 0.95288223],
       [0.58360505, 0.41639492],
       [0.58360505, 0.41639492],
       [0.1300516 , 0.8699484 ]], dtype=float32)

In [21]:
model.predict_classes(x_train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 1, 1, 0, 1, 0, 0, 1])